In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import Policy, Critic, ensemblize

import flax
import flax.linen as nn

class Temperature(nn.Module):
    initial_temperature: float = 1.0

    @nn.compact
    def __call__(self) -> jnp.ndarray:
        log_temp = self.param('log_temp',
                              init_fn=lambda key: jnp.full(
                                  (), jnp.log(self.initial_temperature)))
        return jnp.exp(log_temp)

class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: TrainState
    target_critic: TrainState
    actor: TrainState
    temp: TrainState
    config: dict = nonpytree_field()

    @jax.jit
    def update(agent, batch: Batch):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        def critic_loss_fn(critic_params):
            next_dist = agent.actor(batch['next_observations'])
            next_actions, next_log_probs = next_dist.sample_and_log_prob(seed=next_key)

            next_q1, next_q2 = agent.target_critic(batch['next_observations'], next_actions)
            next_q = jnp.minimum(next_q1, next_q2)
            target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q

            if agent.config['backup_entropy']:
                target_q = target_q - agent.config['discount'] * batch['masks'] * next_log_probs * agent.temp()
            
            q1, q2 = agent.critic(batch['observations'], batch['actions'], params=critic_params)
            critic_loss = ((q1 - target_q)**2 + (q2 - target_q)**2).mean()
            
            return critic_loss, {
                'critic_loss': critic_loss,
                'q1': q1.mean(),
            }        

        def actor_loss_fn(actor_params):
            dist = agent.actor(batch['observations'], params=actor_params)
            actions, log_probs = dist.sample_and_log_prob(seed=curr_key)
            
            q1, q2 = agent.critic(batch['observations'], actions)
            q = jnp.minimum(q1, q2)

            actor_loss = (log_probs * agent.temp() - q).mean()
            return actor_loss, {
                'actor_loss': actor_loss,
                'entropy': -1 * log_probs.mean(),
            }
        
        def temp_loss_fn(temp_params, entropy, target_entropy):
            temperature = agent.temp(params=temp_params)
            temp_loss = (temperature * (entropy - target_entropy)).mean()
            return temp_loss, {
                'temp_loss': temp_loss,
                'temperature': temperature,
            }
        
        new_critic, critic_info = agent.critic.apply_loss_fn(loss_fn=critic_loss_fn, has_aux=True)
        new_target_critic = target_update(agent.critic, agent.target_critic, agent.config['target_update_rate'])
        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=actor_loss_fn, has_aux=True)

        temp_loss_fn = functools.partial(temp_loss_fn, entropy=actor_info['entropy'], target_entropy=agent.config['target_entropy'])
        new_temp, temp_info = agent.temp.apply_loss_fn(loss_fn=temp_loss_fn, has_aux=True)

        return agent.replace(rng=new_rng, critic=new_critic, target_critic=new_target_critic, actor=new_actor, temp=new_temp), {
            **critic_info, **actor_info, **temp_info}

    @jax.jit
    def sample_actions(agent,
                       observations: np.ndarray,
                       *,
                       seed: PRNGKey,
                       temperature: float = 1.0,
                       ) -> jnp.ndarray:
        actions = agent.actor(observations, temperature=temperature).sample(seed=seed)
        actions = jnp.clip(actions, -1, 1)
        return actions



def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 temp_lr: float = 3e-4,
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 tau: float = 0.005,
                 target_entropy: float = None,
                 backup_entropy: bool = True,
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = Policy(hidden_dims, action_dim=action_dim, 
            log_std_min=-10.0, state_dependent_std=True, tanh_squash_distribution=True, final_fc_init_scale=1.0)

        actor_params = actor_def.init(actor_key, observations)['params']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=actor_lr))

        critic_def = ensemblize(Critic, num_qs=2)(hidden_dims)
        critic_params = critic_def.init(critic_key, observations, actions)['params']
        critic = TrainState.create(critic_def, critic_params, tx=optax.adam(learning_rate=critic_lr))
        target_critic = TrainState.create(critic_def, critic_params)

        temp_def = Temperature()
        temp_params = temp_def.init(rng)['params']
        temp = TrainState.create(temp_def, temp_params, tx=optax.adam(learning_rate=temp_lr))

        if target_entropy is None:
            target_entropy = -0.5 * action_dim

        config = flax.core.FrozenDict(dict(
            discount=discount,
            target_update_rate=tau,
            target_entropy=target_entropy,
            backup_entropy=backup_entropy,            
        ))

        return SACAgent(rng, critic=critic, target_critic=target_critic, actor=actor, temp=temp, config=config)

def get_default_config():
    import ml_collections

    return ml_collections.ConfigDict({
        'actor_lr': 3e-4,
        'critic_lr': 3e-4,
        'temp_lr': 3e-4,
        'hidden_dims': (256, 256),
        'discount': 0.99,
        'tau': 0.005,
        'target_entropy': ml_collections.config_dict.placeholder(float),
        'backup_entropy': True,
    })

In [3]:
import os
from functools import partial
import numpy as np
import jax
import tqdm
import gymnasium as gym

import examples.mujoco.sac as learner

from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
import wandb
from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
from jaxrl_m.dataset import ReplayBuffer

#from ml_collections import config_flags
import pickle
#from flax.training import checkpoints


#FLAGS = flags.FLAGS
env_name='Humanoid-v4'
seed=np.random.choice(1000000)
eval_episodes=10
batch_size = 256
max_steps = int(1e6)
start_steps = int(1e4)                     
log_interval = 10000
eval_interval = 10000

wandb_config = default_wandb_config()
wandb_config.update({
    'project': 'd4rl_test',
    'group': 'sac_test',
    'name': 'sac_{env_name}',
})


env = EpisodeMonitor(gym.make(env_name))
eval_env = EpisodeMonitor(gym.make(env_name))
setup_wandb({"bonjour":1})

example_transition = dict(
    observations=env.observation_space.sample(),
    actions=env.action_space.sample(),
    rewards=0.0,
    masks=1.0,
    next_observations=env.observation_space.sample(),
)

replay_buffer = ReplayBuffer.create(example_transition, size=int(1e6))

agent = learner.create_learner(seed,
                example_transition['observations'][None],
                example_transition['actions'][None],
                max_steps=max_steps,
                #**FLAGS.config
                )

exploration_metrics = dict()
obs,info = env.reset()    
exploration_rng = jax.random.PRNGKey(0)

for i in tqdm.tqdm(range(1, max_steps + 1),
                    smoothing=0.1,
                    dynamic_ncols=True):

    if i < start_steps:
        action = env.action_space.sample()
    else:
        exploration_rng, key = jax.random.split(exploration_rng)
        action = agent.sample_actions(obs, seed=key)

    #next_obs, reward, done, info = env.step(action)
    next_obs, reward, done, truncated, info = env.step(action)
    
    mask = float(not done or 'TimeLimit.truncated' in info)
    
    replay_buffer.add_transition(dict(
        observations=obs,
        actions=action,
        rewards=reward,
        masks=mask,
        next_observations=next_obs,
    ))
    obs = next_obs

    if (done or truncated):
        exploration_metrics = {f'exploration/{k}': v for k, v in flatten(info).items()}
        obs,info= env.reset()

    if replay_buffer.size < start_steps:
        continue

    batch = replay_buffer.sample(batch_size)  
    agent, update_info = agent.update(batch)

    if i % log_interval == 0:
        train_metrics = {f'training/{k}': v for k, v in update_info.items()}
        wandb.log(train_metrics, step=i)
        wandb.log(exploration_metrics, step=i)
        exploration_metrics = dict()

    if i % eval_interval == 0:
        
        
        policy_fn = partial(supply_rng(agent.sample_actions), temperature=0.0)
        eval_info = evaluate(policy_fn, eval_env, num_episodes=eval_episodes)
        eval_metrics = {f'evaluation/{k}': v for k, v in eval_info.items()}
        wandb.log(eval_metrics, step=i)

    # if i % FLAGS.save_interval == 0 and FLAGS.save_dir is not None:
    #     checkpoints.save_checkpoint(FLAGS.save_dir, agent, i)



wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


evaluation/episode.duration,█▆▆▄▆▅▆▇▄▇▅▅▆▇▇▅▄▁█▅▇▄▅▃▃▇▆▆▅▅▆▇▇▄▅▆▆▆▄▂
evaluation/episode.length,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
evaluation/episode.return,▁▂▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████▇██████
evaluation/final.episode.duration,█▆▆▄▆▅▆▇▄▇▅▅▆▇▇▅▄▁█▅▇▄▅▃▃▇▆▆▅▅▆▇▇▄▅▆▆▆▄▂
evaluation/final.episode.length,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
evaluation/final.episode.return,▁▂▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████▇██████
evaluation/final.reward_ctrl,█▂▁▂▂▂▂▃▃▃▃▃▃▃▂▃▂▂▄▃▃▃▂▃▂▃▃▄▄▃▃▃▄▃▄▃▃▃▄▄
evaluation/final.reward_run,▁▁▄▄▅▅▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██████
evaluation/final.total.timesteps,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
evaluation/final.x_position,▁▂▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████▇██████
evaluation/final.x_velocity,▁▁▄▄▅▅▅▅▅▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██▇██████


Extra kwargs: {'max_steps': 1000000}


  0%|          | 0/1000000 [00:00<?, ?it/s]/home/mahdi/Desktop/jaxrl_m/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_normalized_score to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_normalized_score` for environment variables or `env.get_wrapper_attr('get_normalized_score')` that will search the reminding wrappers.
  logger.warn(
  1%|          | 9991/1000000 [00:01<03:09, 5221.47it/s]

{'evaluation/reward_linvel': 0.04036260457808817, 'evaluation/reward_quadctrl': -1.5370676755205086, 'evaluation/reward_alive': 5.0, 'evaluation/x_position': 0.01969457311990607, 'evaluation/y_position': 0.012533799396730008, 'evaluation/distance_from_origin': 0.028180817394746163, 'evaluation/x_velocity': 0.032290083662470534, 'evaluation/y_velocity': 0.17864813009504163, 'evaluation/forward_reward': 0.04036260457808817, 'evaluation/total.timesteps': 98.0, 'evaluation/episode.return': 68.3142511166228, 'evaluation/episode.length': 19.5, 'evaluation/episode.duration': 0.019856739044189452, 'evaluation/final.reward_linvel': 0.10087588239553355, 'evaluation/final.reward_quadctrl': -1.584960444662704, 'evaluation/final.reward_alive': 5.0, 'evaluation/final.x_position': 0.027166398218548438, 'evaluation/final.y_position': 0.05070850314084596, 'evaluation/final.distance_from_origin': 0.058067008749369445, 'evaluation/final.x_velocity': 0.08070070591642683, 'evaluation/final.y_velocity': 0.3

...

{'evaluation/reward_linvel': 0.39272573899933566, 'evaluation/reward_quadctrl': -0.6235798508294409, 'evaluation/reward_alive': 5.0, 'evaluation/x_position': 2.248542166535392, 'evaluation/y_position': -0.46149568121050816, 'evaluation/distance_from_origin': 2.3024062565462535, 'evaluation/x_velocity': 0.3141805911994685, 'evaluation/y_velocity': -0.056490118221905854, 'evaluation/forward_reward': 0.39272573899933566, 'evaluation/total.timesteps': 713216.5, 'evaluation/episode.return': 4769.145888169894, 'evaluation/episode.length': 1000.0, 'evaluation/episode.duration': 0.5884100437164307, 'evaluation/final.reward_linvel': 0.4321127350138122, 'evaluation/final.reward_quadctrl': -0.5670292630036305, 'evaluation/final.reward_alive': 5.0, 'evaluation/final.x_position': 4.722133960184426, 'evaluation/final.y_position': -0.8481899082279114, 'evaluation/final.distance_from_origin': 4.805484135738242, 'evaluation/final.x_velocity': 0.3456901880110497, 'evaluation/final.y_velocity': 0.0123371

 99%|█████████▉| 990157/1000000 [27:20<00:50, 195.41it/s]

{'evaluation/reward_linvel': 0.6929311979034711, 'evaluation/reward_quadctrl': -0.8220111701337677, 'evaluation/reward_alive': 5.0, 'evaluation/x_position': 2.728453971210801, 'evaluation/y_position': -0.26720033192523135, 'evaluation/distance_from_origin': 2.7636289635624403, 'evaluation/x_velocity': 0.5543449583227769, 'evaluation/y_velocity': -0.07095295192075414, 'evaluation/forward_reward': 0.6929311979034711, 'evaluation/total.timesteps': 721208.0, 'evaluation/episode.return': 2914.2714526146133, 'evaluation/episode.length': 598.3, 'evaluation/episode.duration': 0.3393065929412842, 'evaluation/final.reward_linvel': 1.518330410689876, 'evaluation/final.reward_quadctrl': -1.096188100540254, 'evaluation/final.reward_alive': 5.0, 'evaluation/final.x_position': 4.994428248688161, 'evaluation/final.y_position': -0.6320540252153176, 'evaluation/final.distance_from_origin': 5.063851228615454, 'evaluation/final.x_velocity': 1.2146643285519008, 'evaluation/final.y_velocity': -0.16777393081

100%|██████████| 1000000/1000000 [27:38<00:00, 603.08it/s]

{'evaluation/reward_linvel': 0.40464280880038034, 'evaluation/reward_quadctrl': -0.637901392495868, 'evaluation/reward_alive': 5.0, 'evaluation/x_position': 2.421232292415809, 'evaluation/y_position': 0.04348681162768253, 'evaluation/distance_from_origin': 2.435754818853999, 'evaluation/x_velocity': 0.32371424704030427, 'evaluation/y_velocity': 0.00988794390738605, 'evaluation/forward_reward': 0.40464280880038034, 'evaluation/total.timesteps': 729199.5, 'evaluation/episode.return': 4766.741416304513, 'evaluation/episode.length': 1000.0, 'evaluation/episode.duration': 0.5694445371627808, 'evaluation/final.reward_linvel': 0.4173464920448233, 'evaluation/final.reward_quadctrl': -0.5976481763812167, 'evaluation/final.reward_alive': 5.0, 'evaluation/final.x_position': 4.8709326853409864, 'evaluation/final.y_position': 0.15007856592094318, 'evaluation/final.distance_from_origin': 4.8955581274684485, 'evaluation/final.x_velocity': 0.33387719363585866, 'evaluation/final.y_velocity': 0.01638288